In [1]:
try:
    with open('i.html') as fp:
        text = fp.read()
except UnicodeDecodeError as exc:
    print(exc)

'utf-8' codec can't decode byte 0x80 in position 1721: invalid start byte


In [2]:
with open('i.html', 'rb') as fp:
    octets = fp.read()

for i in range(1715, 1725):
    code = octets[i]
    octet = octets[i:i+1]
    char = octet.decode('cp1252')
    print(f'{code:02x} {char!r}')

0a '\n'
0a '\n'
3c '<'
21 '!'
2d '-'
2d '-'
80 '€'
80 '€'
80 '€'
80 '€'


In [3]:
with open('i.html', encoding='cp1252') as fp:
     text = fp.read()

In [17]:
import re
import html

hexagram_start = re.compile(r'<a name="\d{1,2}"></a>')
title_parts = re.compile(r'(\d{1,2})\. ([^\n]+)')
chapters = hexagram_start.split(html.unescape(text))
for i, chapter in enumerate(chapters[1:], 1):
    verses = chapter.split('\n')
    hit = title_parts.search(verses[1])
    n, title = hit.groups(verses[1])
    title_wg, title_en = title.split(' / ')
    assert i == int(n)
    print(f'{n:2}\t{title_wg:10} {title_en}')

1 	Ch'ien     The Creative
2 	K'un       The Receptive
3 	Chun       Difficulty at the Beginning
4 	Mêng       Youthful Folly
5 	Hsü        Waiting (Nourishment)
6 	Sung       Conflict
7 	Shih       The Army
8 	Pi         Holding Together [union]
9 	Hsiao Ch'u The Taming Power of the Small
10	Lü         Treading [conduct]
11	T'ai       Peace
12	P'i        Standstill [Stagnation]
13	T'ung Jên  Fellowship with Men
14	Ta Yu      Possession in Great Measure
15	Ch'ien     Modesty
16	Yü         Enthusiasm
17	Sui        Following
18	Ku         Work on what has been spoiled [ Decay ]
19	Lin        Approach
20	Kuan       Contemplation (View)
21	 Shih Ho   Biting Through
22	Pi         Grace
23	Po         Splitting Apart
24	Fu         Return (The Turning Point)
25	Wu Wang    Innocence (The Unexpected)
26	Ta Ch'u    The Taming Power of the Great
27	I          Corners of the Mouth (Providing Nourishment)
28	Ta Kuo     Preponderance of the Great
29	K'an       The Abysmal (Water)
30	Li         The Cl